In [1]:
import keras
import keras.backend as K
import numpy as np
import tensorflow as tf
from sklearn.model_selection import ParameterGrid

import pbt
from pbt.utils import train_population

Using TensorFlow backend.


In [2]:
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [3]:
VALIDATION_SPLIT = 0.3
# population_size = 20
BATCH_SIZE = 32  # one step in a member is equivalent to train on a batch
TOTAL_STEPS = 1000  # total number of steps before ending training
STEPS_READY = 50  # number of steps before updating a member
STEPS_SAVE = 100  # number of steps before saving the status of the population

In [4]:
# ------------------------------------------
# SEARCH SPACE
# ------------------------------------------
l1_values = np.linspace(1e-5, 0.01, num=6).tolist()
l2_values = np.linspace(1e-5, 0.01, num=6).tolist()
param_grid = ParameterGrid(dict(l1=l1_values, l2=l2_values))

In [5]:
def build_fn(data_dim, l1=1e-5, l2=1e-5):
    """Returns a function which can be called with no parameters to create a new
     model.

    The returned function is required by Member, which invokes it to construct a
    new instance of the model.

    Returns:
        function: a function that can be called to get a compiled Keras model.

    """

    def _build_fn():
        # Set seed to get same weight initialization
        np.random.seed(42)
        model = keras.models.Sequential([
            keras.layers.Dense(64,
                               activation='relu',
                               input_shape=(data_dim,),
                               kernel_regularizer=
                               pbt.hyperparameters.l1_l2(l1, l2)),
            keras.layers.Dense(1,
                               kernel_regularizer=
                               pbt.hyperparameters.l1_l2(l1, l2)),
        ])
        model.compile(optimizer='adam', loss='mean_squared_error')
        return model

    return _build_fn

In [6]:
def show_results(df):
    print('***** RESULTS *****')
    population_size = df['model_id'].nunique()
    df_final = df.tail(population_size)
    print('** Ranking **')
    print(df_final[['model_id', 'loss']].sort_values('loss'))
    print('** Statistics **')
    print(df_final['loss'].describe())
    print('** Best hyperparameters **')
    best = df.iloc[[df_final['loss'].idxmin()]]
    print(best.filter(regex=".+:.+"))

In [7]:
# Load dataset. Split training dataset into train and validation sets
dataset = tf.keras.datasets.boston_housing
(x_train, y_train), (x_test, y_test) = dataset.load_data()
print('Train examples: {}, test examples: {}'.format(
    x_train.shape[0], x_test.shape[0]
))
data_dim = x_train.shape[1]

Train examples: 404, test examples: 102


In [8]:
# ------------------------------------------
# GRID SEARCH
# ------------------------------------------

# We create a population where we never explore new hyperparameters
population = []
for h in param_grid:
    member = pbt.members.Member(build_fn(data_dim, **h),
                                steps_ready=None)
    population.append(member)

res_gd = train_population(population, x_train, y_train, BATCH_SIZE,
                          TOTAL_STEPS, STEPS_SAVE, VALIDATION_SPLIT)
show_results(res_gd)

1000/1000 [==============================] - 306s 306ms/step - min: 22.0262 - max: 35.8625 - mean: 25.9173
***** RESULTS *****
** Ranking **
            model_id       loss
330  139625074369928  22.026169
339  139625061307616  22.858138
336  139625064962424  22.881347
342  139625055506504  23.264763
337  139625063467160  23.534788
338  139625062724944  23.546447
348  139625048515304  23.553898
354  139625040971144  23.619396
349  139625047109816  23.628204
355  139625040098528  23.870604
341  139625056924280  24.253792
345  139625051547800  24.356365
331  139625070617824  24.534729
344  139625053047160  24.703714
350  139625045696584  24.735127
359  139625033231776  24.866910
343  139625054444960  25.013111
356  139625036591848  25.088423
357  139625035707000  25.110988
333  139625068323448  25.166521
324  139625183648232  25.223250
332  139625069732584  25.626381
351  139625045147040  25.793447
352  139625043745144  25.891963
325  139625183711184  26.161741
358  139625034285128  26.25

In [9]:
# ------------------------------------------
# PBT
# ------------------------------------------
K.clear_session()

# This time, the members will explore new hyperparameters
population = []
for h in param_grid:
    member = pbt.members.Member(build_fn(data_dim, **h),
                                steps_ready=STEPS_READY)
    population.append(member)

res_pbt = train_population(population, x_train, y_train, BATCH_SIZE,
                           TOTAL_STEPS, STEPS_SAVE, VALIDATION_SPLIT)
show_results(res_pbt)

1000/1000 [==============================] - 428s 428ms/step - min: 21.7823 - max: 21.9551 - mean: 21.8081
***** RESULTS *****
** Ranking **
            model_id       loss
357  139624648720848  21.782267
356  139624649610072  21.782267
355  139624651019600  21.782267
348  139624659432280  21.782267
347  139624660845904  21.782267
353  139624653277504  21.782267
349  139624658026960  21.782267
352  139624654670312  21.782267
344  139624664504808  21.782269
351  139624656076304  21.782269
350  139624657154232  21.782270
345  139624663095616  21.782270
337  139624672397632  21.782272
334  139624676282552  21.782272
343  139624665378320  21.782273
346  139624661734680  21.782274
339  139624670143824  21.782278
328  139624683072424  21.782282
338  139624671556888  21.782285
340  139624668738392  21.782296
341  139624667849168  21.782303
330  139624840994152  21.782305
342  139624666452152  21.782323
332  139624678679384  21.782336
336  139624673802728  21.782373
331  139624679699400  21.78